## Librerías

In [1]:
# Librerías de ciencia de datos 
import numpy as np
import pandas as pd
import sqlite3 as sql

# para análisis interactivo
from ipywidgets import interact, Output, widgets

# Librerías para preprocesamiento 
from sklearn.preprocessing import MinMaxScaler
from sklearn import neighbors 
import joblib

# Paquete para sistemas de recomendación surprise 
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise import Reader, Dataset

## Carga de datos

In [2]:
#### Conectar #######
conn=sql.connect('data/db_movies')
cur=conn.cursor()

In [3]:
#ver tablas
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
cur.fetchall()

[('ratings',),
 ('movies',),
 ('usuarios_sel',),
 ('movies_sel',),
 ('ratings_final',),
 ('movie_final',),
 ('full_ratings',),
 ('full_ratings_dum',),
 ('reco',)]

In [4]:
# Vamos a trabajar con el dataframe escalado y con dummies df_dum 
df_dum = pd.read_sql("SELECT * FROM full_ratings_dum", conn)
movies = pd.read_sql("SELECT * FROM movie_final", conn)

# Se selecciona un usuario para realizar las recomendaciones 
usuarios = pd.read_sql('SELECT distinct (userId) as user_id from ratings_final', conn)

# Seleccionamos un usuario para realizar el sistema de recomendación
user_id = 605
movies
ratings = pd.read_sql('SELECT * FROM ratings_final', conn)

## 3. Sistema de recomendación basado en contenido KNN Con base en todo lo visto por el usuario

### Top 10 de las películas más similares a las vistas por el usuario y con una que le haya dado una calificación mayor a 3.5.

In [14]:

# Función para recomendar películas
def recomendar(user_id, k=5):
    
    """ 
    Esta función tiene como objetivo recomendar películas a un usuario específico en función de sus calificaciones. 
    Para ello, recibe como argumento el user_id, que identifica al usuario, y un parámetro opcional k, 
    que determina la cantidad de recomendaciones a realizar (por defecto, 5).
    
    La función comienza obteniendo las calificaciones del usuario desde una base de datos y filtra aquellas películas 
    que tienen un rating superior a 3.5. Si el usuario no tiene calificaciones válidas, la función devuelve un mensaje indicándolo. 
    A continuación, calcula un "centroide" que representa las preferencias del usuario en función de las películas que ha calificado. 
    Luego, utiliza el modelo de K-Vecinos más cercanos (KNN) para identificar películas similares que el usuario no ha visto. 
    Finalmente, retorna un dataframe con los títulos de las películas recomendadas.
    
    """
 # Obtener las calificaciones del usuario
    ratings = pd.read_sql('select * from ratings_final where userId=:user', conn, params={'user': user_id})
    
    # Filtrar películas con rating mayor a 3.5
    ratings = ratings[ratings['rating'] > 3.5]
    
    # Si no hay ratings válidos, devolver mensaje
    if ratings.empty:
        return f"El usuario {user_id} no tiene películas calificadas con un rating mayor a 3.5."

    # Convertir ratings a array
    ratings_array = ratings['movieId'].to_numpy()
    
    # Agregar movieId y título al dataframe df_dum
    df_dum[['movieId', 'title']] = movies[['movieId', 'title']]
    
    # Filtrar películas calificadas por el usuario
    movies_r = df_dum[df_dum['movieId'].isin(ratings_array)]
    
    # Calcular el centroide del usuario
    movies_r = movies_r.drop(columns=['movieId', 'title'])
    movies_r["indice"] = 1
    centroide = movies_r.groupby("indice").mean()
    
    # Filtrar películas no vistas por el usuario
    movies_ns = df_dum[~df_dum['movieId'].isin(ratings_array)].drop(columns=['movieId', 'title'])
    
    # Entrenar el modelo de K-Vecinos
    model = neighbors.NearestNeighbors(n_neighbors=k, metric='cosine')
    model.fit(movies_ns)
    
    # Obtener las películas más cercanas
    dist, idlist = model.kneighbors(centroide)
    
    # Comprobar si hay suficientes vecinos cercanos
    if len(idlist[0]) < k:
        return f"No hay suficientes vecinos cercanos para el usuario {user_id}."

    # Obtener los títulos de las películas recomendadas
    ids = idlist[0]
    recomend_m = movies.loc[ids][['title']]
    
    return recomend_m


# Función para la interfaz interactiva
def interfaz_recomendaciones(user_id):
    """ 
    Esta función tiene como objetivo proporcionar una interfaz interactiva para mostrar recomendaciones de películas a un usuario específico. 
    Recibe como argumento el user_id, que identifica al usuario cuyas preferencias de películas se están considerando.

    Dentro de la función, se llama a la función recomendar, pasando el user_id como parámetro para obtener las recomendaciones correspondientes. 
    Luego, imprime un mensaje que indica el usuario para el cual se están generando las recomendaciones, seguido de la lista de películas recomendadas.
    """
    recomendaciones_resultado = recomendar(user_id)
    print(f"Recomendaciones para el usuario {user_id}:")
    print(recomendaciones_resultado)

# Crear un widget interactivo para que el usuario seleccione el user_id
interact(interfaz_recomendaciones,
         user_id=widgets.IntSlider(min=1, max=1000, step=1, value=605, description='User ID:'))


interactive(children=(IntSlider(value=605, description='User ID:', max=1000, min=1), Output()), _dom_classes=(…

<function __main__.interfaz_recomendaciones(user_id)>

### 3.1. Top 5 de las películas más similares a las vistas por el usuario según el género

In [6]:
# Limpiar la columna 'genres': eliminar espacios extra y caracteres no deseados
movies['genres'] = movies['genres'].str.strip()

# Crear un widget Output para controlar las salidas
output = Output()

In [13]:


# Función para recomendar películas por género seleccionado y mostrar el rating promedio
def recomendar_peliculas_por_genero(genero_seleccionado, user_id, conn, top_n=5):
    
    """ 
    Esta función recomienda películas a un usuario según un género específico. Recibe como argumentos el genero_seleccionado, 
    el user_id, la conexión conn a la base de datos y un parámetro opcional top_n para definir cuántas recomendaciones se deben devolver (por defecto es 5).
    Filtra las películas que el usuario ha visto y las que no, seleccionando solo aquellas del género elegido. 
    Calcula el rating promedio de las películas recomendadas y retorna los títulos y ratings de las top_n mejores opciones.
    """
    
    # Obtener ratings del usuario
    ratings_user = pd.read_sql('select * from ratings_final where userId=:user', conn, params={'user': user_id})

    # Obtener películas vistas por el usuario
    ratings_array = ratings_user['movieId'].to_numpy()
    peliculas_vistas = movies[movies['movieId'].isin(ratings_array)]

    # Expandir géneros de las películas vistas
    peliculas_vistas.loc[:, 'genres'] = peliculas_vistas['genres'].str.split('|')
    peliculas_con_generos = peliculas_vistas.explode('genres')

    # Filtrar películas no vistas del género seleccionado
    peliculas_no_vistas = movies[~movies['movieId'].isin(ratings_array)]
    genero_seleccionado_str = str(genero_seleccionado).strip()
    peliculas_recomendadas = peliculas_no_vistas[peliculas_no_vistas['genres'].str.contains(genero_seleccionado_str, regex=False)]

    # Calcular rating promedio para las películas
    rating_promedio = ratings.groupby('movieId')['rating'].mean().reset_index()
    peliculas_recomendadas = peliculas_recomendadas.merge(rating_promedio, on='movieId', how='left')

    # Redondear ratings y seleccionar las mejores
    peliculas_recomendadas['rating'] = peliculas_recomendadas['rating'].round(2)
    top_recomendadas = peliculas_recomendadas[['title', 'rating']].head(top_n)

    return top_recomendadas

# Obtener todos los géneros únicos de la columna 'genres'
todos_los_generos = movies['genres'].str.split('|').explode().unique()

# Crear la función interactiva con salida controlada


@interact
def mostrar_recomendaciones(genero=todos_los_generos):
    output.clear_output()  # Limpiar la salida anterior
    recomendaciones = recomendar_peliculas_por_genero(genero, user_id, conn)
    with output:
        print(f"Recomendaciones para el género '{genero}':")
        print(recomendaciones)

# Mostrar el widget de salida
display(output)


interactive(children=(Dropdown(description='genero', options=('Adventure', 'Animation', 'Children', 'Comedy', …

Output(outputs=({'name': 'stdout', 'text': "Recomendaciones para el género 'Adventure':\n                     …

## 4. Sistema de recomendación filtro colaborativo

## Top 10 de las películas con mayor calificación predicha

In [18]:

# Obtener datos de calificaciones, filtrando las mayores a 0
rat = pd.read_sql('select * from ratings_final where rating > 0', conn)

# Definir escala de calificaciones
reader = Reader(rating_scale=(1, 5))

# Cargar datos en el formato esperado por Surprise
data = Dataset.load_from_df(rat[['userId', 'movieId', 'rating']], reader)

### Selección del modelo

In [19]:
# Modelos a probar
models = [KNNBasic(), KNNWithMeans(), KNNWithZScore(), KNNBaseline()]
results = {}

In [20]:
# Evaluar modelos usando validación cruzada
for model in models:
    CV_scores = cross_validate(model, data, measures=["MAE", "RMSE"], cv=5, n_jobs=-1)
    result = pd.DataFrame.from_dict(CV_scores).mean(axis=0).rename({'test_mae': 'MAE', 'test_rmse': 'RMSE'})
    results[str(model).split("algorithms.")[1].split("object ")[0]] = result


In [33]:
# DataFrame con resultados de rendimiento
performance_df = pd.DataFrame.from_dict(results).T.sort_values(by='RMSE')
performance_df

,MAE,RMSE,fit_time,test_time
knns.KNNBaseline,0.630826,0.830077,0.274264,2.409560
knns.KNNWithZScore,0.634585,0.835972,0.287830,1.646991
knns.KNNWithMeans,0.639064,0.839044,0.266684,1.433566
knns.KNNBasic,0.668206,0.881025,0.191487,1.270799


In [22]:
# Selección del mejor modelo
selected_model = KNNBasic

### Afinamiento de hiperparámetros

In [25]:
param_grid = {
    'sim_options': {
        'name': ['msd', 'cosine'],
        'user_based': [False, True],
        'min_support': [1, 5, 10]
    },
    'k': [5, 10, 15],  # Número de vecinos
}


### Búsqueda de hiperparámetros

In [27]:
# Búsqueda de hiperparámetros
gridsearch = GridSearchCV(selected_model, param_grid, measures=['rmse'], cv=2, n_jobs=-1)
gridsearch.fit(data)

In [37]:
# Mejor modelo y parámetros
best_params = gridsearch.best_params["rmse"]
best_score = gridsearch.best_score["rmse"]
gs_model = gridsearch.best_estimator['rmse']

In [29]:
# Entrenamiento con todos los datos y predicciones
trainset = data.build_full_trainset()
model1 = gs_model.fit(trainset)
predset = trainset.build_anti_testset()
predictions = gs_model.test(predset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [30]:
# DataFrame de predicciones
predictions_df = pd.DataFrame(predictions)
predictions_df.head()

,uid,iid,r_ui,est,details
0,1,21,3.717143,4.045181,"{'actual_k': 15, 'was_impossible': False}"
1,1,32,3.717143,4.679153,"{'actual_k': 15, 'was_impossible': False}"
2,1,357,3.717143,4.409305,"{'actual_k': 15, 'was_impossible': False}"
3,1,368,3.717143,4.473641,"{'actual_k': 15, 'was_impossible': False}"
4,1,509,3.717143,4.443098,"{'actual_k': 15, 'was_impossible': False}"


In [34]:

# Función para recomendar películas
def recomendaciones(user_id, n_recomend=11):
    predictions_userID = predictions_df[predictions_df['uid'] == user_id] \
                         .sort_values(by="est", ascending=False).head(n_recomend)
    recomendados = predictions_userID[['iid', 'est']]
    recomendados.to_sql('reco', conn, if_exists="replace")

    # Obtener títulos de películas recomendadas
    mov = pd.read_sql('''SELECT b.title, a.est AS estimated_rating 
                         FROM reco a 
                         LEFT JOIN movie_final b ON a.iid = b.movieId''', conn)
    mov['estimated_rating'] = mov['estimated_rating'].round(2)
    
    return mov

In [35]:
# ID del usuario para las recomendaciones
user_id = 608
n_recomend = 10

peliculas = recomendaciones(user_id, n_recomend)
peliculas

,title,estimated_rating
0,Heat (1995),4.25
1,Sense and Sensibility (1995),4.15
2,Raising Arizona (1987),4.03
3,Life Is Beautiful (La Vita è bella) (1997),4.02
4,American Psycho (2000),4.00
5,Shakespeare in Love (1998),3.97
6,In the Line of Fire (1993),3.96
7,Cool Hand Luke (1967),3.95
8,"Dark Knight, The (2008)",3.91
9,Little Miss Sunshine (2006),3.90
